In [ ]:
import pandas as pd
from pathlib import Path

DATA_PATH = 'data'

file_name = (
    Path('tables_OZ_geo_5500') /
    'Ozon_Crawler_Latest_info2025-04-07-12-57-51.xlsx'
)

source_df = pd.read_excel(Path(DATA_PATH) / file_name)
source_df.head(1)

,Sku (Sku),Фото (CoverImage),Название товара (ProductName),Продавец (SellerName),Бренд (BrandName),Название категории (CategoryName),Цена соинвест (DiscountPrice),Цена по карте (OzonCardPrice),Сток FBO (StockFbm),Сток FBS (StockFbs),Ошибка загрузки (CrawlerError),Валюта (Currency),Цена до скидки (BasePrice),Рейтинг товара (AvgRating),Количество отзывов (Reviews),Описание (Description),Rich-контент (RichContent),Ссылка на карточку,SellerProductId (SellerProductId)
0,1871769771,https://cdn1.ozone.ru/s3/multimedia-1-b/732378...,"Карты МИРА и РОССИИ настенные политические,160...",ИНТЕРТРЕЙД,Атлас Принт,Карты мира,1785.0,1649,0,34,0,RUB,3867.0,5.0,1,Представляем вашему вниманию уникальный набор ...,NaN,https://www.ozon.ru/product/karty-mira-i-rossi...,191942573


In [ ]:
# Collect images

import nest_asyncio
nest_asyncio.apply()

from utils import parallel_download_img

# Dataset name
DATASET_NAME = "images_OZ_geo_5500"

# Download image files using the 'Thumb' column for both URL and ID extraction
results = await parallel_download_img(
    df=source_df,
    url_col='Фото (CoverImage)',
    img_id_regex=r'/(\d+)\.jpg$',  # capture numeric ID before '.jpg'
    dataset_name=DATASET_NAME,
    base_download_dir=DATA_PATH,
    max_concurrency=100,
    delay=1.0
)

# Merge the list of results into your DataFrame as a new column

In [9]:
source_df

,Sku (Sku),Фото (CoverImage),Название товара (ProductName),Продавец (SellerName),Бренд (BrandName),Название категории (CategoryName),Цена соинвест (DiscountPrice),Цена по карте (OzonCardPrice),Сток FBO (StockFbm),Сток FBS (StockFbs),Ошибка загрузки (CrawlerError),Валюта (Currency),Цена до скидки (BasePrice),Рейтинг товара (AvgRating),Количество отзывов (Reviews),Описание (Description),Rich-контент (RichContent),Ссылка на карточку,SellerProductId (SellerProductId)
0,1871769771,https://cdn1.ozone.ru/s3/multimedia-1-b/732378...,"Карты МИРА и РОССИИ настенные политические,160...",ИНТЕРТРЕЙД,Атлас Принт,Карты мира,1785.0,1649,0,34,0,RUB,3867.0,5.0,1,Представляем вашему вниманию уникальный набор ...,NaN,https://www.ozon.ru/product/karty-mira-i-rossi...,191942573
1,1679550303,https://cdn1.ozone.ru/s3/multimedia-1-5/739430...,Схема линий скоростного транспорта Москвы (Мет...,ИНТЕРТРЕЙД,Атлас Принт,Карты мира,528.0,487,14,91,0,RUB,913.0,4.8,620,Схема линий скоростного транспорта Москвы ваш ...,NaN,https://www.ozon.ru/product/shema-liniy-skoros...,191942573
2,1200553001,https://cdn1.ozone.ru/s3/multimedia-1-8/729902...,"Политическая карта МИРА 160х109 см, Карта мира...",ИНТЕРТРЕЙД,Атлас Принт,Карты мира,1906.0,1761,0,1,0,RUB,3404.0,4.8,1519,Настенная карта мира — универсальное учебное п...,NaN,https://www.ozon.ru/product/politicheskaya-kar...,191942573
3,922231521,https://cdn1.ozone.ru/s3/multimedia-1-u/738853...,"Политическая карта МИРА настенная, 100х70см, ш...",ИНТЕРТРЕЙД,Атлас Принт,Карты мира,587.0,541,19,20,0,RUB,1319.0,4.8,1519,Политическая карта мира настенная в ТУБУСЕ — и...,NaN,https://www.ozon.ru/product/politicheskaya-kar...,191942573
4,922230517,https://cdn1.ozone.ru/s3/multimedia-1-3/729507...,"Политическая карта МИРА настенная, 160х102см, ...",ИНТЕРТРЕЙД,Атлас Принт,"Гравюры, литография и карты",888.0,822,53,44,0,RUB,2204.0,4.8,1519,Настенная карта мира — уникальное сочетание об...,NaN,https://www.ozon.ru/product/politicheskaya-kar...,191942573
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5560,166584090,https://cdn1.ozone.ru/s3/multimedia-x/60085388...,Карта настенная Мир. Политическая карта с флаг...,Самсон,DMB,"Гравюры, литография и карты",410.0,398,20,0,0,RUB,823.0,4.9,879,"Настенная карта ""Мир. Политическая карта с фла...",NaN,https://www.ozon.ru/product/karta-nastennaya-m...,191942573
5561,166451882,https://cdn1.ozone.ru/s3/multimedia-f/60084386...,"Карта мира, магнитно-маркерная, 60х40 см, комп...",Art-board,АртБорд,"Гравюры, литография и карты",5466.0,5042,0,15,0,RUB,6800.0,5.0,5,"Физическая карта мира, магнитно-маркерная. На ...",NaN,https://www.ozon.ru/product/karta-mira-magnitn...,191942573
5562,154409524,https://cdn1.ozone.ru/s3/multimedia-1-d/743954...,Карта звездного неба космоса настенная светяща...,Gift Development,Gift Development,Карты мира,657.0,606,52,515,0,RUB,1188.0,4.9,390,Представляем вашему вниманию настенную светящу...,NaN,https://www.ozon.ru/product/karta-zvezdnogo-ne...,191942573
5563,147896031,https://cdn1.ozone.ru/s3/multimedia-1-x/709834...,Скретч карта мира географическая Dark Edition ...,Gift Development,Gift Development,Карты мира,731.0,674,9,377,0,RUB,1500.0,4.9,1058,"Страны, которые вы посетили, и путешествия, ко...",NaN,https://www.ozon.ru/product/skretch-karta-mira...,191942573


In [ ]:
# Save images

source_df['image_name'] = results
source_df.to_csv(Path(DATA_PATH) / 'OZ_geo_5500.csv', index=None)